In [1]:
from bayes_opt import BayesianOptimization
from L1_DifferentialEvolution import extract_data, model, diff_evolution
from scipy.optimize import Bounds
from scipy.optimize import minimize
from NelderMead import bounded_nelder_mead
from losses import Losses
import math
import physics
import numpy as np
import matplotlib.pyplot as plt

In [2]:
frequency, intensity = extract_data() # extracting the desired information

# compressor parameters - obtained as minus the stretcher one - TO BE FIXED WITH REAL ONES ONCE THEY ARE AVAILABLE
COMPRESSOR = -1 * np.array((267.422 * 1e-24, -2.384 * 1e-36, 9.54893 * 1e-50)) # in s^2, s^3 and s^4 (SI units)
# non linearity parameter
B = 2
# cutoff frequencies, in THz, used to remove noise - derived from visual inspection
CUTOFF = (289.95, 291.91)
# model instantiation
l1_pump = model(frequency, intensity, COMPRESSOR, B, CUTOFF, num_points=int(5e3))

losses = Losses(l1_pump)

bounds_DE = Bounds(
            # GDD         # TOD          # FOD
    lb = (2.3522e-22, -1.003635e-34, 4.774465e-50),
    ub = (2.99624e-22, 9.55955e-35, 1.4323395e-49)
)

bounds_matrix = np.vstack((bounds_DE.lb, bounds_DE.ub)).T
disp_bounds = np.sort(l1_pump.translate_control(bounds_matrix, verse = "to_disp"))

In [3]:
def outer_objfunc(mutation:float, cross_p:float, population_size:float) -> float: 
    """
    This is the outher objective function which is taking as input values characteristic of the 
    optimisation process.
    For further documentantion on the relevance of these parameters and their effect on the optimisation 
    route check literature.
    """
    population_size = int(population_size) # must be int
    
    candidate_best = diff_evolution(
        objective_function = losses.loss3,
        bounds = disp_bounds,
        mutation = mutation, 
        cross_p = cross_p, 
        population_size = population_size,
        verbose = 0, 
        maxit = 20, 
        print_every = 50,
        print_NM_progress = 0
    )
    
    return -1 * losses.loss3(candidate_best) # maximizing

In [ ]:
params_bounds = {
    "mutation": (0.5, 1.99), 
    "cross_p": (0, 1), 
    "population_size": (10, 50)
}

optimizer = BayesianOptimization(
    f = outer_objfunc, 
    pbounds = params_bounds, 
    random_state = 10, 
    verbose = 1
)

n_init, n_iter = 50, 50
optimizer.maximize(init_points = n_init, n_iter = n_iter)
print(optimizer.max["params"].values())
# dict_values([0.4418331744229961, 1.1466808500666077, 34.71067913877269])

In [7]:
cross_p = 0.4418331744229961
mutation = 1.1466808500666077
pop_size = 34.71067913877269

# training with the best parameters and increased number of iterations
candidate_best, fev = diff_evolution(
        objective_function = losses.loss3,
        bounds = disp_bounds,
        mutation = mutation, 
        cross_p = cross_p, 
        population_size = int(pop_size),
        verbose = 1, 
        maxit = 12, 
        print_every = 1,
        print_NM_progress = 1
)

loss = losses.loss3(candidate_best)

print(loss)
print(candidate_best)

Iteration 0 - Objective Function value: 33.25252713980094
Iteration 1 - Objective Function value: 33.25252713980094
Iteration 2 - Objective Function value: 26.013976629009093
Iteration 3 - Objective Function value: 26.013976629009093
Iteration 4 - Objective Function value: 26.013976629009093
iteration: 1
best value of objective function so far: 25.572770042883118
best candidate: [-4.72433508e-01  1.83030532e+06  1.56859005e+15]
iteration: 2
best value of objective function so far: 25.572770042883118
best candidate: [-4.72433508e-01  1.83030532e+06  1.56859005e+15]
iteration: 3
best value of objective function so far: 25.572770042883118
best candidate: [-4.72433508e-01  1.83030532e+06  1.56859005e+15]
iteration: 4
best value of objective function so far: 25.572770042883118
best candidate: [-4.72433508e-01  1.83030532e+06  1.56859005e+15]
iteration: 5
best value of objective function so far: 25.572770042883118
best candidate: [-4.72433508e-01  1.83030532e+06  1.56859005e+15]
iteration: 6

iteration: 5
best value of objective function so far: 1.4606605107884654
best candidate: [-4.75425713e-01 -6.93661166e+06  1.40892082e+15]
iteration: 6
best value of objective function so far: 1.4606605107884654
best candidate: [-4.75425713e-01 -6.93661166e+06  1.40892082e+15]
iteration: 7
best value of objective function so far: 1.4606605107884654
best candidate: [-4.75425713e-01 -6.93661166e+06  1.40892082e+15]
iteration: 8
best value of objective function so far: 1.4606605107884654
best candidate: [-4.75425713e-01 -6.93661166e+06  1.40892082e+15]
iteration: 9
best value of objective function so far: 1.4606605107884654
best candidate: [-4.75425713e-01 -6.93661166e+06  1.40892082e+15]
iteration: 10
best value of objective function so far: 1.4606605107884654
best candidate: [-4.75425713e-01 -6.93661166e+06  1.40892082e+15]
iteration: 11
best value of objective function so far: 1.4606605107884654
best candidate: [-4.75425713e-01 -6.93661166e+06  1.40892082e+15]
iteration: 12
best value 

Iteration 9 - Objective Function value: 1.443329509589482
Iteration 10 - Objective Function value: 1.443329509589482
iteration: 1
best value of objective function so far: 1.443329509589482
best candidate: [-4.75472822e-01 -6.95115709e+06  1.56859005e+15]
iteration: 2
best value of objective function so far: 1.443329509589482
best candidate: [-4.75472822e-01 -6.95115709e+06  1.56859005e+15]
iteration: 3
best value of objective function so far: 1.443329509589482
best candidate: [-4.75472822e-01 -6.95115709e+06  1.56859005e+15]
iteration: 4
best value of objective function so far: 1.443329509589482
best candidate: [-4.75472822e-01 -6.95115709e+06  1.56859005e+15]
iteration: 5
best value of objective function so far: 1.443329509589482
best candidate: [-4.75472822e-01 -6.95115709e+06  1.56859005e+15]
iteration: 6
best value of objective function so far: 1.443329509589482
best candidate: [-4.75472822e-01 -6.95115709e+06  1.56859005e+15]
iteration: 7
best value of objective function so far: 1